In [ ]:
# default_exp utils

# Utils

> contains various util functions and classes 

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import re

import pandas as pd
import numpy as np
from random import randrange

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler
%matplotlib inline

ModuleNotFoundError: No module named 'memory_profiler'

In [ ]:
#export
def f1score(truth, classified):
    tp = len(set(truth).intersection(set(classified)))
    t = len(truth)
    p = len(classified)
    
    if (p == 0):
         precision = 1
    else:
        precision = tp/p
        
    if (t == 0):
        recall = 1
    else:
        recall = tp/t
    
    if (precision == 0 and recall == 0):
        f1 = 0
    else:
        f1 = 2*precision*recall/(precision+recall)
    return f1, tp, t, p

In [ ]:
#export
def df_preproc (df):
    df['event_id']= df.index
    df.index = df['trace_id']
    df = df[["event_id", "activity", "trace_id"]]
    
    for i in df['trace_id'].unique():
        df_cop = df.loc[i]
        df.loc[i, 'event_id']  = np.arange(1, len(df_cop)+1)
        
    df.reset_index(drop=True, inplace=True)
    
    trace_ends = list(df.loc[df['event_id']==1].index)[1:]
    trace_ends.append(len(df))
    new = np.insert(np.array(df),trace_ends, [-1, 'end', 0], axis=0)
    df = pd.DataFrame(data=new, columns=["event_id", "activity", "trace_id"])
    
    trace_starts = list(df.loc[df['event_id']==1].index)
    new = np.insert(np.array(df),trace_starts, [0, 'start', 0], axis=0)
    
    trace_starts = np.where(new[:,0]==0)[0]
    trace_ends = np.where(new[:,0]==-1)[0]
    new[trace_starts,2] = new[np.array(trace_starts)+1,2]
    new[trace_ends,2] = new[np.array(trace_starts)+1,2]
    
    
    df = pd.DataFrame(data=new, columns=["event_id", "activity", "trace_id"])
    df.index = df['trace_id']

    return df

In [ ]:
def load_data (data='PDC2016'):
    '''loads eventlogs from different sources, returns event_df with start/end event, test_df and truth_df'''
    
    #---------------PDC2016|2017--------------------------------------------------
    
    if (data == 'PDC2016' or data == 'PDC2017'):
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train'
        files = [filename for filename in os.listdir(directory) if filename.split('.')[-1] =='csv']
        files.sort()
        pick = randrange(len(files)) #picks random eventlog
        df = pd.read_csv('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train/' + files[pick], sep=',')
        df = df.rename(columns={"act_name": "activity", "case_id": "trace_id"})
        event_df = df_preproc(df)
        
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test'
        files = [filename for filename in os.listdir(directory)]
        files.sort()
        log = xes_importer.apply('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test/' + files[pick])
        df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
        df = df.rename(columns={"concept:name": "activity", "case:concept:name": "trace_id"})
        df = df[['activity', 'trace_id']]
        test_df = df_preproc(df)
        
        truth = [log[i].attributes['pdc:isPos'] for i in range(len(log))]
        d = {'case': range(1, len(log)+1), 'normal': truth}
        truth_df = pd.DataFrame (data=d)
    
    #---------------PDC2019--------------------------------------------------
    
    if (data == 'PDC2019'):
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train'
        files = [filename for filename in os.listdir(directory) if filename.split('.')[-1] =='csv']
        files.sort()
        pick = randrange(len(files)) #picks random eventlog
        df = pd.read_csv('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/train/' + files[pick], sep=',')
        df = df.rename(columns={"event": "activity", "case": "trace_id"})
        df = df[['activity', 'trace_id']]
        event_df = df_preproc(df)
        
    
        directory = r'/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test'
        files = [filename for filename in os.listdir(directory)]
        files.sort()
        log = xes_importer.apply('/mnt/data/jlahann/jupyterlab/fastpm/nbs/PDC data/'+ data +'/test/' + files[pick])
        df = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
        df = df.rename(columns={"concept:name": "activity", "case:concept:name": "trace_id"})
        df = df[['activity', 'trace_id']]
        test_df = df_preproc(df)
        
        truth = [log[i].attributes['pdc:isPos'] for i in range(len(log))]
        d = {'case': range(1, len(log)+1), 'normal': truth}
        truth_df = pd.DataFrame (data=d)
        
        
    #---------------PDC2020--------------------------------------------------
        
    
    
    return event_df, test_df, truth_df